In [36]:
import glob
import numpy as np
from particle import Particle
import os

`path_templ = "/home/maximilian/corsika/modules/data/Pythia/pythia8311_xsec/main1010_{projectile}_{target}_1e{logE}_*.dat"`

In [37]:
path_templ = "/home/gaudu/PYTHIA/pythia8312/examples/xsec_pythia8312/main1011_{projectile}_{target}_1e{logE}_10000.dat"

In [38]:
projectiles = {
    #"p": 2212,
    #"n": 2112,
    #"ap": -2212,
    #"an": -2112,
    #"pi0": 111,
    #"pip": 211,
    #"pim": -211,
    #"k0": 311,
    #"kp": 321,
    #"km": -321,
    #"k0s": 310,
    #"k0l": 130,
    #"lambda0": 3122,
    #"sigma0": 3212,
    #"sigmam": 3112,
    #"sigmap": 3222,
    #"xi0": 3322
    "4He": 1000020040,
    "14N": 1000070140,
    "56Fe": 1000260560
}

targets = {
    #"1H": 2212,
    #"12C": 1000060120,
    "14N": 1000070140
    #"16O": 1000080160,
    #"40Ar": 1000180400
}

proj_pdg_to_str = dict((v, k) for k, v in projectiles.items())
target_pdg_to_str = dict((v, k) for k, v in targets.items())

energies = range(2, 13)

In [39]:
def format_path(proj, targ, logE):
    return path_templ.format(projectile=proj_pdg_to_str[proj], target=target_pdg_to_str[targ], logE=str(logE))

In [40]:
dtype = np.dtype([
    ("elab", np.float32),
    ("sig_tot", np.float32),
    ("err_tot", np.float32),
    ("sig_nd", np.float32),
    ("err_nd", np.float32),
    ("sig_el", np.float32),
    ("err_el", np.float32),
    ("sig_XB", np.float32),
    ("err_XB", np.float32),
    ("sig_AX", np.float32),
    ("err_AX", np.float32),
    ("sig_XX", np.float32),
    ("err_XX", np.float32),
    ("sig_inel", np.float32),
    ("err_inel", np.float32)
])

In [45]:
particle_data = {}
for proj_pdg in projectiles.values():
    projectile = Particle.from_pdgid(proj_pdg)
    for target_pdg in targets.values():
        table = np.zeros(len(energies), dtype=dtype)
        for i, logE in enumerate(energies):
            path = format_path(proj_pdg, target_pdg, logE)
            #print(path)
            gl = glob.glob(path)
            if not gl:
                print(f"No files found for path: {path}")
                continue
            with open(next(iter(gl)), "rt") as f:
                data = dict()
                for l in f:
                    key, val = l.split()
                    try:
                        data[key] = int(val)
                    except ValueError:
                        data[key] = float(val)
            #print(f"Target ID in file: {data['targ_id']}, Expected: {target_pdg}")
            #print(f"Projectile ID in file: {data['proj_id']}, Expected: {proj_pdg}")
            assert(data["targ_id"] == target_pdg)
            assert(data["proj_id"] == proj_pdg)
            for key, val in data.items():
                if key in dtype.fields:
                    table[i][key] = float(val)
        particle_data[(proj_pdg), (target_pdg)] = table
        d = {key: particle_data[(proj_pdg), (target_pdg)][key] for key in dtype.fields}
        np.savez(f"xs_{proj_pdg}_{target_pdg}.npz", **d)


In [42]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [43]:
with PdfPages("xs.pdf") as pdf:
    for (projectile, target), dataset in particle_data.items():
        fig, ax = plt.subplots(figsize=(4,3))
        ax.set(xscale="log")
        for field in dtype.fields:
            if not field.startswith("e"):
                ax.plot(dataset['elab'], dataset[field], "o-", label=str(field))
        ax.text(.5, .98, f"(${Particle.from_pdgid(projectile).latex_name}$:${target}$)",
                ha="center", va="top", transform=ax.transAxes)
        ax.legend(loc="upper left", fontsize="small", handlelength=1.2)
        fig.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

In [44]:
{key: particle_data[(proj_pdg), (target_pdg)][key] for key in dtype.fields}

{'elab': array([1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08, 1.e+09,
        1.e+10, 1.e+11, 1.e+12], dtype=float32),
 'sig_tot': array([1827.48, 1934.97, 1995.21, 2068.45, 2172.48, 2291.38, 2378.78,
        2544.83, 2665.23, 2834.39, 2990.7 ], dtype=float32),
 'err_tot': array([0.297098, 0.310011, 0.320508, 0.331972, 0.346118, 0.363489,
        0.378215, 0.400676, 0.420227, 0.445466, 0.469545], dtype=float32),
 'sig_nd': array([1698.88, 1751.44, 1811.63, 1859.65, 1965.39, 2088.43, 2191.52,
        2341.45, 2460.61, 2603.97, 2775.49], dtype=float32),
 'err_nd': array([0.3012  , 0.315671, 0.324952, 0.337798, 0.351406, 0.368761,
        0.382734, 0.405168, 0.424607, 0.450893, 0.474018], dtype=float32),
 'sig_el': array([0.444022, 0.      , 0.58291 , 0.      , 0.300565, 0.267956,
        1.20511 , 1.03149 , 2.29647 , 4.12175 , 5.21079 ], dtype=float32),
 'err_el': array([25.1251,  0.    , 32.5423,  0.    , 47.0804, 41.7899, 23.6771,
        30.8462, 24.6301, 17.5908, 17.3412], 